# TermProject

2019102093 소프트웨어융합학과 명승준

# 주제 : 우리 가족이 외식갈 때 모두가 만족하는 음식을 먹으려면 ?

## (1) 주제 선정 이유

 중간고사가 끝나고 오랜만에 집에 갔다. 집에서 쉬면서 Termproject에 대해 생각하고 있었는데, Termproject를 통해 처음으로 하는 data 분석인 만큼 의미있는 분석을 하자고 생각하는 와중에 마침 부모님과 형이 어디로 외식을 가실지 고민하시는 모습을 보았다. 따라서, 우리 집 주변의 음식점 중 하나를 기호에 맞게 추천하는 프로그램을 만들고 외식 갈 때 실제로 써보려 한다.

## (2) 가설 정의


 가족 구성원은 아버지, 어머니, 형, 그리고 나 총 4명이다. 구성원 각각은 가장 먹고 싶은 음식 종류와 가장 먹기 싫은 음식 종류를 입력할 수 있다.(먹고 싶은 음식과 먹기 싫은 음식 둘 다 공란으로 남길 수도 있다.) 구성원에게 타이핑받은 음식 종류는 점수를 부여받는다. 특히, 가족 구성원의 지위에 의거하여 점수는 차등화된다. (ex: 어머니의 Best/Worst = ±2, 아버지의 선호 = ±2, 형의 선호 = ±1.5, 나의 선호 = ±1) 선호를 입력 받은 후 프로그램은 가장 높은 점수를 받은 음식 종류를 보여주고, 그 종류에 해당하는 음식점을 밑에 나열한다. 각 음식점마다 이름, 전화번호, 위치가 나타난다. "그 날의 이벤트"를 입력할 수 있다. 예컨대, 형의 생일이라면 형이 부여할 수 있는 점수가 높아진다.(1.5 -> 2.5) 비 오는 날이라면, 김치전이나 메기탕 등의 메뉴를 파는 음식점의 기본 점수가 높아진다.

## (3) 인터넷을 통한 data 획득 

- 서울특별시 광진구 모범음식점 (https://www.data.go.kr/dataset/3054109/fileData.do)
- 경기도 남양주시 모범음식점 (https://www.data.go.kr/dataset/3045042/fileData.do)

우리 가족은 주로 외식 갈 때 광진구 ~ 남양주시를 벗어나지 않으므로, 광진구와 남양주시의 음식점 정보를 공공데이터포탈에서 가져왔다.

## (4) 분석을 위한 data의 가공

두 data를 하나로 만든다.


(시행착오 :
#1 : 오픈 API에서 가져온 csv 파일을 그대로 사용할 때 문제점은, pd.read_csv로 가져오는 dataframe의 열 구분이 사라진다는 것이다. 예컨대 dataframe의 columns가 ['업소명', '종류', '도로명주소', '전화번호','주메뉴'] 로 나타나야 하는데, ['업소명, 종류, 도로명주소, 전화번호, 주메뉴'] 로 단 하나의 elements를 갖는 List 형태가 나타났다. .split()을 이용하여 elements를 쪼개려고 시도했으나, 행렬 자체가 N * 1 행렬이므로 쪼개기를 포기하였다. csv 파일을 사용할 때 파일이 손상될 수 있다는 경고에 근거하여, csv 파일을 사용한 것 때문에 이와 같은 현상이 발생했다고 판단하고, 파일을 excel 서식 파일로 바꾸어 pd.read_excel을 이용하였다.
#2 : df_GJ의 columns는 ['업소명', '업태명', '도로명주소', '전화번호', '주메뉴', '데이터기준일'] 이고, df_NYJ의 columns는 ['업소명', '업태명', '주소', '주메뉴', '전화번호 ','시도명', '시군구명'] 이다. 두 데이터를 하나로 합치고 싶으므로 pd.concat을 이용할 예정이므로 columns의 이름을 통일했다. (df_GJ의 '데이터기준일' 삭제 및 '업태명' -> '종류' 변경, df_NYJ의 '시도명', '시군구명' 삭제 및 '업태명' -> '종류' 변경 및 '주소' -> '도로명주소' 변경 및 '전화번호 ' -> '전화번호' 변경)
#3 : #2까지의 과정을 거치고 pd.concat 을 이용해 데이터를 합쳐서 316 * 5 행렬을 예상했다. 그런데 index가 0~ 159, 0~ 156 으로 지정되어있었다. 겹치는 index가 있으면 나중에 slicing 할 때 문제가 있을 수 있으므로 df_NYJ의 index를 157만큼 미루었다.)

In [1]:
# 두 data를 하나로 합치기
import pandas as pd

df_GJ = pd.read_excel('GJ.xltx', encoding = 'ANSI') # 광진구 음식점 data                                     #!
df_GJ.rename(columns = {df_GJ.columns[1] : "종류"}, inplace = True) #data 합치기 위한 준비
del df_GJ['데이터기준일']

df_NYJ = pd.read_excel('NYJ.xltx', encoding = 'ANSI') # 남양주 음식점 data
del df_NYJ['시도명']
del df_NYJ['시군구명']
df_NYJ.rename(columns = {df_NYJ.columns[3] : "도로명주소"}, inplace = True)
df_NYJ.rename(columns = {df_NYJ.columns[0] : "종류"}, inplace = True)
df_NYJ.rename(columns = {df_NYJ.columns[4] : "전화번호"}, inplace = True)                                    #2
for i in range(159):
    df_NYJ.rename(index = {df_NYJ.index[i] : i+157} , inplace = True) # data 합치기 위한 준비                #3

Data = pd.concat([df_GJ, df_NYJ], sort = False) # 데이터 합치기
for i in range(316):
    if Data.iloc[i,1] == "중국식":
        Data.iloc[i,1] = "중식"  #중복되는 카테고리 제거
Data

,업소명,종류,도로명주소,전화번호,주메뉴
0,부림정1,한식,서울특별시 광진구 뚝섬로 476-7 (자양동),02-463-5455,갈비
1,(주)장군갈비,한식,서울특별시 광진구 용마산로 11 (중곡동),02-457-3241,갈비탕
2,거보주,한식,"서울특별시 광진구 아차산로51길 24, 1~2층 (구의동)",02-456-2266,"돼지갈비, 삼겹살"
3,송림식당,한식,"서울특별시 광진구 자양번영로 77 (자양동,,137,138,139,141)",02-457-5473,불고기
4,능동아구탕,한식,서울특별시 광진구 천호대로 544 (군자동),02-467-9350,아구탕
5,송화양꼬치,중식,서울특별시 광진구 동일로18길 70 (자양동),02-462-7826,양꼬치
6,다도회집,일식,서울특별시 광진구 뚝섬로 670 (자양동),02-453-7653,생선회
7,다락,일식,서울특별시 광진구 광나루로 440 (화양동),02-444-4128,알탕
8,제주뜰향갈치전문점,한식,서울특별시 광진구 능동로 387 (중곡동),02-464-6118,갈치조림
9,유일설렁탕,한식,서울특별시 광진구 용마산로 154 (중곡동),02-452-5860,설렁탕


빅 데이터의 특성상 음식 종류를 다 셀 수 없기 때문에, 음식 종류의 개수를 세기 위해 다음의 행동을 취한다.

In [2]:
# 음식 종류 알아내기

FoodSort = set() # 집합의 성질을 이용하여 중복 없애기
for i in range(316):
    FoodSort.add(Data.iloc[i,1])
print(FoodSort)

{'중식', '뷔페식', '기타', '한식', '일식', '호프/통닭', '양식', '분식', '경양식'}


이제 data에 존재하는 모든 음식 종류를 알 수 있으므로, 음식 종류에 따라 음식점을 분류할 수 있다.

In [3]:
# 음식 종류로 음식점 분리하기

KoreaR = []
ChinaR = []
PubR = []
SnackbarR = []
WesternR = []
JapanR = []
etcR = []
LightwesternR = []
VuffetR = []
Res = KoreaR + ChinaR + PubR + SnackbarR + WesternR + JapanR + etcR + LightwesternR + VuffetR

for i in range(316):
    Sort = Data.iloc[i,1]
    restaurant_name = Data.iloc[i,0]
    if Sort == "한식":
        KoreaR.append(restaurant_name)
    elif Sort == "중식":
        ChinaR.append(restaurant_name)
    elif Sort == "호프/통닭":
        PubR.append(restaurant_name)
    elif Sort == "분식":
        SnackbarR.append(restaurant_name)
    elif Sort == "양식":
        WesternR.append(restaurant_name)
    elif Sort == "기타":
        etcR.append(restaurant_name)
    elif Sort == "일식":
        JapanR.append(restaurant_name)
    elif Sort == "경양식":
        LightwesternR.append(restaurant_name)
    elif Sort == "뷔페식":
        VuffetR.append(restaurant_name)

음식 종류에 따라 음식점을 분류하였다. 이제 가족 구성원이 고르는 음식 종류에 따라 점수를 부여하는 메커니즘을 짤 수 있다.

## (5) 분석 결과 도출

가족 구성원들이 선택하는 가장 먹고 싶은 음식 종류 / 가장 먹기 싫은 음식 종류에 따라 점수를 부여하는 메커니즘을 짜려고 한다. 


In [4]:
KoreaS = 0 # 음식 종류를 입력받는 인터페이스
ChinaS = 0
PubS = 0
SnackbarS = 0
WesternS = 0
JapanS = 0
etcS = 0
LightwesternS = 0
VuffetS = 0 # 음식종류의 점수 부여

print("음식 종류 : ", FoodSort)

DadB = input("아버지가 오늘 가장 드시고 싶은 음식 종류는? : ")
DadW = input("아버지가 오늘 가장 드시기 싫은 음식 종류는? : ")
Dad = [DadB,DadW]

MomB = input("어머니가 오늘 가장 드시고 싶은 음식 종류는? : ")
MomW = input("어머니가 오늘 가장 드시기 싫은 음식 종류는? : ")
Mom = [MomB,MomW]

BroB = input("형이 오늘 가장 먹고 싶은 음식 종류는? : ")
BroW = input("형이 오늘 가장 먹기 싫은 음식 종류는? : ")
Bro = [BroB,BroW]

MeB = input("내가 오늘 가장 먹고 싶은 음식 종류는? : ")
MeW = input("내가 오늘 가장 먹기 싫은 음식 종류는? : ")
Me = [MeB,MeW]

SpecialCondition = input("누구의 생일인가요 ?")

음식 종류 :  {'중식', '뷔페식', '기타', '한식', '일식', '호프/통닭', '양식', '분식', '경양식'}
아버지가 오늘 가장 드시고 싶은 음식 종류는? : 한식
아버지가 오늘 가장 드시기 싫은 음식 종류는? : 양식
어머니가 오늘 가장 드시고 싶은 음식 종류는? : 한식
어머니가 오늘 가장 드시기 싫은 음식 종류는? : 뷔페식
형이 오늘 가장 먹고 싶은 음식 종류는? : 일식
형이 오늘 가장 먹기 싫은 음식 종류는? : 분식
내가 오늘 가장 먹고 싶은 음식 종류는? : 양식
내가 오늘 가장 먹기 싫은 음식 종류는? : 한식
누구의 생일인가요 ?아버지


In [5]:
# 가중치 짜기
personValue = 3   #아버지의 가중치

if SpecialCondition == "아버지":
    personValue *= 2

if Dad[0] == "한식":
    KoreaS += personValue
elif Dad[0] == "중식":
    ChinaS += personValue
elif Dad[0] == "호프/통닭":
    PubS += personValue
elif Dad[0] == "분식":
    SnackbarS += personValue
elif Dad[0] == "양식":
    WesternS += personValue
elif Dad[0] == "기타":
    etcS += personValue
elif Dad[0] == "일식":
    JapanS += personValue
elif Dad[0] == "경양식":
    LightwesternS += personValue
elif Dad[0] == "뷔페식":
    VuffetS += personValue

if Dad[1] == "한식":
    KoreaS -= personValue
elif Dad[1] == "중식":
    ChinaS -= personValue
elif Dad[1] == "호프/통닭":
    PubS -= personValue
elif Dad[1] == "분식":
    SnackbarS -= personValue
elif Dad[1] == "양식":
    WesternS -= personValue
elif Dad[1] == "기타":
    etcS -= personValue
elif Dad[1] == "일식":
    JapanS -= personValue
elif Dad[1] == "경양식":
    LightwesternS -= personValue
elif Dad[1] == "뷔페식":
    VuffetS -= personValue
    
personValue = 2   #어머니의 가중치

if SpecialCondition == "어머니":
    personValue *= 2

if Mom[0] == "한식":
    KoreaS += personValue
elif Mom[0] == "중식":
    ChinaS += personValue
elif Mom[0] == "호프/통닭":
    PubS += personValue
elif Mom[0] == "분식":
    SnackbarS += personValue
elif Mom[0] == "양식":
    WesternS += personValue
elif Mom[0] == "기타":
    etcS += personValue
elif Mom[0] == "일식":
    JapanS += personValue
elif Mom[0] == "경양식":
    LightwesternS += personValue
elif Mom[0] == "뷔페식":
    VuffetS += personValue

if Mom[1] == "한식":
    KoreaS -= personValue
elif Mom[1] == "중식":
    ChinaS -= personValue
elif Mom[1] == "호프/통닭":
    PubS -= personValue
elif Mom[1] == "분식":
    SnackbarS -= personValue
elif Mom[1] == "양식":
    WesternS -= personValue
elif Mom[1] == "기타":
    etcS -= personValue
elif Mom[1] == "일식":
    JapanS -= personValue
elif Mom[1] == "경양식":
    LightwesternS -= personValue
elif Mom[1] == "뷔페식":
    VuffetS -= personValue
    
personValue = 1.5   #형의 가중치

if SpecialCondition == "형":
    personValue *= 2

if Bro[0] == "한식":
    KoreaS += personValue
elif Bro[0] == "중식":
    ChinaS += personValue
elif Bro[0] == "호프/통닭":
    PubS += personValue
elif Bro[0] == "분식":
    SnackbarS += personValue
elif Bro[0] == "양식":
    WesternS += personValue
elif Bro[0] == "기타":
    etcS += personValue
elif Bro[0] == "일식":
    JapanS += personValue
elif Bro[0] == "경양식":
    LightwesternS += personValue
elif Bro[0] == "뷔페식":
    VuffetS += personValue

if Bro[1] == "한식":
    KoreaS -= personValue
elif Bro[1] == "중식":
    ChinaS -= personValue
elif Bro[1] == "호프/통닭":
    PubS -= personValue
elif Bro[1] == "분식":
    SnackbarS -= personValue
elif Bro[1] == "양식":
    WesternS -= personValue
elif Bro[1] == "기타":
    etcS -= personValue
elif Bro[1] == "일식":
    JapanS -= personValue
elif Bro[1] == "경양식":
    LightwesternS -= personValue
elif Bro[1] == "뷔페식":
    VuffetS -= personValue
    
personValue = 1   #나의 가중치

if SpecialCondition == "나":
    personValue *= 2

if Me[0] == "한식":
    KoreaS += personValue
elif Me[0] == "중식":
    ChinaS += personValue
elif Me[0] == "호프/통닭":
    PubS += personValue
elif Me[0] == "분식":
    SnackbarS += personValue
elif Me[0] == "양식":
    WesternS += personValue
elif Me[0] == "기타":
    etcS += personValue
elif Me[0] == "일식":
    JapanS += personValue
elif Me[0] == "경양식":
    LightwesternS += personValue
elif Me[0] == "뷔페식":
    VuffetS += personValue

if Me[1] == "한식":
    KoreaS -= personValue
elif Me[1] == "중식":
    ChinaS -= personValue
elif Me[1] == "호프/통닭":
    PubS -= personValue
elif Me[1] == "분식":
    SnackbarS -= personValue
elif Me[1] == "양식":
    WesternS -= personValue
elif Me[1] == "기타":
    etcS -= personValue
elif Me[1] == "일식":
    JapanS -= personValue
elif Me[1] == "경양식":
    LightwesternS -= personValue
elif Me[1] == "뷔페식":
    VuffetS -= personValue

In [6]:
# 점수 합산 및 계산하여 순위 만들기
Dic_between_Food_Score = {"한식" : KoreaS, "중식" : ChinaS, "호프/통닭" : PubS, "분식" : SnackbarS, "양식" : WesternS, "일식" : JapanS, "기타" : etcS, "경양식" : LightwesternS, "뷔페식" : VuffetS}
Ranking = sorted(Dic_between_Food_Score, key = lambda k : Dic_between_Food_Score[k])
Scores = sorted(Dic_between_Food_Score.values())

firstR = Ranking.pop()
secondR = Ranking.pop()
thirdR = Ranking.pop()

firstS = Scores.pop()
secondS = Scores.pop()
thirdS = Scores.pop()

if firstS != 0:
    print("음식 종류 추천 1위 : {} (점수 : {})".format(firstR, firstS))
elif firstS == 0:
    print("모두 0점. 우열을 가릴 수 없습니다.")
if secondS != 0:
    print("음식 종류 추천 2위 : {} (점수 : {})".format(secondR, secondS))
if thirdS != 0:
    print("음식 종류 추천 3위 : {} (점수 : {})".format(thirdR, thirdS))

음식 종류 추천 1위 : 한식 (점수 : 7)
음식 종류 추천 2위 : 일식 (점수 : 1.5)


In [7]:
# 음식 종류에 따른 음식점 음식 분류하기
KoreaFs = set()
ChinaFs = set()
PubFs = set()
SnackbarFs = set()
WesternFs = set()
JapanFs = set()
etcFs = set()
LightwesternFs = set()
VuffetFs = set()

for i in range(316):
    if Data.iloc[i,1] == "한식":
        KoreaFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "중식":
        ChinaFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "호프/통닭":
        PubFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "분식":
        SnackbarFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "양식":
        WesternFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "일식":
        JapanFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "기타":
        etcFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "경양식":
        LightwesternFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "뷔페식":
        VuffetFs.add(Data.iloc[i,4])

KoreaF = []
ChinaF = []
PubF = []
SnackbarF = []
WesternF = []
JapanF = []
etcF = []
LightwesternF = []
VuffetF = []

for i in KoreaFs:
    KoreaF.append(i)
for i in ChinaFs:
    ChinaF.append(i)
for i in PubFs:
    PubF.append(i)
for i in SnackbarFs:
    SnackbarF.append(i)
for i in WesternFs:
    WesternF.append(i)
for i in JapanFs:
    JapanF.append(i)
for i in etcFs:
    etcF.append(i)
for i in LightwesternFs:
    LightwesternF.append(i)
for i in VuffetFs:
    VuffetF.append(i)

In [8]:
ResName = []

def Search(Choose, df):
    for i in range(316):
        if Choose == df.iloc[i,4]:
            ResName.append(df.iloc[i,0])
    
if firstR == "한식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(KoreaF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "중식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(ChinaF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "호프/통닭":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(PubF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "분식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(SnackbarF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "양식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(WesternF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "기타":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(etcF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "일식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(JapanF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "경양식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(LightwesternF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "뷔페식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(VuffetF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
    
if secondR == "한식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(KoreaF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "중식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(ChinaF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "호프/통닭":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(PubF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "분식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(SnackbarF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "양식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(WesternF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "기타":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(etcF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "일식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(JapanF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "경양식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(LightwesternF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "뷔페식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(VuffetF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
    
if thirdR == "한식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(KoreaF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "중식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(ChinaF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "호프/통닭":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(PubF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "분식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(SnackbarF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "양식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(WesternF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "기타":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(etcF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "일식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(JapanF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "경양식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(LightwesternF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "뷔페식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(VuffetF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)    

1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : ['전복 한정식', '매운탕. 장어구이. 백숙', '아구탕', '오리구이', '등심. 돼지갈비', '국밥. 등심모듬', '쌀국수', '찜', '청국장', '삼겹살', '간장게장', '회냉면', '소곱창', '구이류. 한정식', '도토리', '납곱전골', '육개장', '갈치조림', '두부요리', '능이버섯불고기', '소머리국밥', '생선회', '오리훈제', '냉면', '약초닭백숙', '숯불고기', '곤드레밥. 청국장', '불고기', '유기농쌈밥. 구이류', '비빔국수', '삼겹살구이', '소고기. 돼지고기 구이류', '오리고기', '냉면. 만두. 불고기', '유기농가마솥밥', '쌈밥', '보쌈. 족발', '나능이버섯백숙', '치킨', '해물찜, 바지락칼국수', '소갈비살', '돼지고기구이', '쭈꾸미', '곤드레정식', '숯불갈비', '보쌈,족발', '꽃등심', '두부전골', '이야기정식', '초당순두부. 짬뽕순두부', '수제비', '돼지갈비', '설농탕', '돼지고기, 쭈꾸미', '매운탕. 장어구이', '불고기냉면', '백숙', '샤브샤브', '순대국', '콩요리정식', '한우등심', '아구찜. 바다가재. 해물짐', '한정식. 팥죽', '감자탕', '곰탕', '갈비구이', '오리구리장작구이. 산꼼장어', '능이버섯백숙. 삼계탕', '해물찜', '칼국수', '한우생고기', '샌드위치', '영양돌솥밥', '보리밥', '등심. 삼겹살', '낙지전골, 갈비전골', '김밥', '돼지갈비. 등심', '흑돈생삼겹살. 가마솥해장국', '만두전골', '홍삼합', '구이류', '쌈밥정식. 보쌈정식', '등심. 왕생갈비', '매운탕', '소갈비. 돼지갈비. 게장', '솥밥한정식', '갈비살', '갈비', '삼계탕', '아구찜', '코다리찜', '김치찌개', '장어구이', '한정식', '해물순두부', '버섯불고기', '등심', '해장국', '국수전골', '돼지왕갈비', '두부', '돼지갈비, 삼겹살', '뽕나무동충하초삼계탕. 영

## (6) 결론 

Data의 도로명주소들로부터 구글 스프레드시트의 Geocode by Awesome Table을 이용하여 위도/경도 값을 구했다. 이를 새 엑셀 파일로 저장하고, pandas를 이용하여 Address라는 이름으로 파일을 연다. 이제 추천하는 음식점 중 무작위 한 곳의 위치를 folium을 이용하여 나타내겠다.

In [9]:
Address = pd.read_excel('Res_Address.xlsx', encoding = "ANSI")

import folium
import random

name = random.choice(firstR)
k = firstR.index(name)
lat = Address.iloc[k,1]
lng = Address.iloc[k,2]

Map_of_Res = folium.Map(location = [lat, lng], zoom_start = 17)
folium.Marker([lat, lng], popup = name).add_to(Map_of_Res)
Map_of_Res

In [ ]:
name = random.choice(secondR)
k = secondR.index(name)
lat = Address.iloc[k,1]
lng = Address.iloc[k,2]

Map_of_Res = folium.Map(location = [lat, lng], zoom_start = 17)
folium.Marker([lat, lng], popup = name).add_to(Map_of_Res)
Map_of_Res

In [ ]:
name = random.choice(thirdR)
k = thirdR.index(name)
lat = Address.iloc[k,1]
lng = Address.iloc[k,2]

Map_of_Res = folium.Map(location = [lat, lng], zoom_start = 17)
folium.Marker([lat, lng], popup = name).add_to(Map_of_Res)
Map_of_Res

## (7) 참고문헌

- "파이썬을 이용한 빅데이터 수집, 분석과 시각화 페이스북 트위터 네이버 공공 일반 웹 데이터 기반", 이원하 지음 | 비팬북스 | 2017년 06월 20일 출간
- "파이썬으로 데이터 주무르기 독특한 예제를 통해 배우는 데이터 분석 입문", 민형기 지음, 비제이퍼블릭, 2017년 12월 29일 출간

## (8) 별첨1 : (3)의 획득한 데이터 원본

In [ ]:
with open('GJread.csv', 'r') as data_GJ:
    print(data_GJ.read())

In [ ]:
with open('NYJread.csv', 'r') as data_NYJ:
    print(data_NYJ.read())

## (9) 별첨2 : (4)의 가공된 데이터 원본

In [ ]:
import pandas as pd

df_GJ = pd.read_excel('GJ.xltx', encoding = 'ANSI')                                   
df_GJ.rename(columns = {df_GJ.columns[1] : "종류"}, inplace = True)
del df_GJ['데이터기준일']

df_NYJ = pd.read_excel('NYJ.xltx', encoding = 'ANSI')

del df_NYJ['시도명']
del df_NYJ['시군구명']
df_NYJ.rename(columns = {df_NYJ.columns[3] : "도로명주소"}, inplace = True)
df_NYJ.rename(columns = {df_NYJ.columns[0] : "종류"}, inplace = True)
df_NYJ.rename(columns = {df_NYJ.columns[4] : "전화번호"}, inplace = True)                              
for i in range(159):
    df_NYJ.rename(index = {df_NYJ.index[i] : i+157} , inplace = True)               

Data = pd.concat([df_GJ, df_NYJ], sort = False)
Data

## (10) 별첨3 : Python 소스코드 원본

import pandas as pd

df_GJ = pd.read_excel('GJ.xltx', encoding = 'ANSI')                                      
df_GJ.rename(columns = {df_GJ.columns[1] : "종류"}, inplace = True) 
del df_GJ['데이터기준일']

df_NYJ = pd.read_excel('NYJ.xltx', encoding = 'ANSI') 
del df_NYJ['시도명']
del df_NYJ['시군구명']
df_NYJ.rename(columns = {df_NYJ.columns[3] : "도로명주소"}, inplace = True)
df_NYJ.rename(columns = {df_NYJ.columns[0] : "종류"}, inplace = True)
df_NYJ.rename(columns = {df_NYJ.columns[4] : "전화번호"}, inplace = True)                                    
for i in range(159):
    df_NYJ.rename(index = {df_NYJ.index[i] : i+157} , inplace = True)                 

Data = pd.concat([df_GJ, df_NYJ], sort = False) 
Data

FoodSort = set() 
for i in range(316):
    FoodSort.add(Data.iloc[i,1])
print(FoodSort)

for i in range(316):
    if Data.iloc[i,1] == "중국식":
        Data.iloc[i,1] = "중식"
        
KoreaR = []
ChinaR = []
PubR = []
SnackbarR = []
WesternR = []
JapanR = []
etcR = []
LightwesternR = []
VuffetR = []

for i in range(316):
    Sort = Data.iloc[i,1]
    restaurant_name = Data.iloc[i,0]
    if Sort == "한식":
        KoreaR.append(restaurant_name)
    elif Sort == "중식":
        ChinaR.append(restaurant_name)
    elif Sort == "호프/통닭":
        PubR.append(restaurant_name)
    elif Sort == "분식":
        SnackbarR.append(restaurant_name)
    elif Sort == "양식":
        WesternR.append(restaurant_name)
    elif Sort == "기타":
        etcR.append(restaurant_name)
    elif Sort == "일식":
        JapanR.append(restaurant_name)
    elif Sort == "경양식":
        LightwesternR.append(restaurant_name)
    elif Sort == "뷔페식":
        VuffetR.append(restaurant_name)
        
KoreaS = 0 
ChinaS = 0
PubS = 0
SnackbarS = 0
WesternS = 0
JapanS = 0
etcS = 0
LightwesternS = 0
VuffetS = 0 

print("음식 종류 : ", FoodSort)

DadB = input("아버지가 오늘 가장 드시고 싶은 음식 종류는? : ")
DadW = input("아버지가 오늘 가장 드시기 싫은 음식 종류는? : ")
Dad = [DadB,DadW]

MomB = input("어머니가 오늘 가장 드시고 싶은 음식 종류는? : ")
MomW = input("어머니가 오늘 가장 드시기 싫은 음식 종류는? : ")
Mom = [MomB,MomW]

BroB = input("형이 오늘 가장 먹고 싶은 음식 종류는? : ")
BroW = input("형이 오늘 가장 먹기 싫은 음식 종류는? : ")
Bro = [BroB,BroW]

MeB = input("내가 오늘 가장 먹고 싶은 음식 종류는? : ")
MeW = input("내가 오늘 가장 먹기 싫은 음식 종류는? : ")
Me = [MeB,MeW]

SpecialCondition = input("누구의 생일인가요 ?")

personValue = 3   

if SpecialCondition == "아버지":
    personValue *= 2

if Dad[0] == "한식":
    KoreaS += personValue
elif Dad[0] == "중식":
    ChinaS += personValue
elif Dad[0] == "호프/통닭":
    PubS += personValue
elif Dad[0] == "분식":
    SnackbarS += personValue
elif Dad[0] == "양식":
    WesternS += personValue
elif Dad[0] == "기타":
    etcS += personValue
elif Dad[0] == "일식":
    JapanS += personValue
elif Dad[0] == "경양식":
    LightwesternS += personValue
elif Dad[0] == "뷔페식":
    VuffetS += personValue

if Dad[1] == "한식":
    KoreaS -= personValue
elif Dad[1] == "중식":
    ChinaS -= personValue
elif Dad[1] == "호프/통닭":
    PubS -= personValue
elif Dad[1] == "분식":
    SnackbarS -= personValue
elif Dad[1] == "양식":
    WesternS -= personValue
elif Dad[1] == "기타":
    etcS -= personValue
elif Dad[1] == "일식":
    JapanS -= personValue
elif Dad[1] == "경양식":
    LightwesternS -= personValue
elif Dad[1] == "뷔페식":
    VuffetS -= personValue
    
personValue = 2   

if SpecialCondition == "어머니":
    personValue *= 2

if Mom[0] == "한식":
    KoreaS += personValue
elif Mom[0] == "중식":
    ChinaS += personValue
elif Mom[0] == "호프/통닭":
    PubS += personValue
elif Mom[0] == "분식":
    SnackbarS += personValue
elif Mom[0] == "양식":
    WesternS += personValue
elif Mom[0] == "기타":
    etcS += personValue
elif Mom[0] == "일식":
    JapanS += personValue
elif Mom[0] == "경양식":
    LightwesternS += personValue
elif Mom[0] == "뷔페식":
    VuffetS += personValue

if Mom[1] == "한식":
    KoreaS -= personValue
elif Mom[1] == "중식":
    ChinaS -= personValue
elif Mom[1] == "호프/통닭":
    PubS -= personValue
elif Mom[1] == "분식":
    SnackbarS -= personValue
elif Mom[1] == "양식":
    WesternS -= personValue
elif Mom[1] == "기타":
    etcS -= personValue
elif Mom[1] == "일식":
    JapanS -= personValue
elif Mom[1] == "경양식":
    LightwesternS -= personValue
elif Mom[1] == "뷔페식":
    VuffetS -= personValue
    
personValue = 1.5   

if SpecialCondition == "형":
    personValue *= 2

if Bro[0] == "한식":
    KoreaS += personValue
elif Bro[0] == "중식":
    ChinaS += personValue
elif Bro[0] == "호프/통닭":
    PubS += personValue
elif Bro[0] == "분식":
    SnackbarS += personValue
elif Bro[0] == "양식":
    WesternS += personValue
elif Bro[0] == "기타":
    etcS += personValue
elif Bro[0] == "일식":
    JapanS += personValue
elif Bro[0] == "경양식":
    LightwesternS += personValue
elif Bro[0] == "뷔페식":
    VuffetS += personValue

if Bro[1] == "한식":
    KoreaS -= personValue
elif Bro[1] == "중식":
    ChinaS -= personValue
elif Bro[1] == "호프/통닭":
    PubS -= personValue
elif Bro[1] == "분식":
    SnackbarS -= personValue
elif Bro[1] == "양식":
    WesternS -= personValue
elif Bro[1] == "기타":
    etcS -= personValue
elif Bro[1] == "일식":
    JapanS -= personValue
elif Bro[1] == "경양식":
    LightwesternS -= personValue
elif Bro[1] == "뷔페식":
    VuffetS -= personValue
    
personValue = 1   

if SpecialCondition == "나":
    personValue *= 2

if Me[0] == "한식":
    KoreaS += personValue
elif Me[0] == "중식":
    ChinaS += personValue
elif Me[0] == "호프/통닭":
    PubS += personValue
elif Me[0] == "분식":
    SnackbarS += personValue
elif Me[0] == "양식":
    WesternS += personValue
elif Me[0] == "기타":
    etcS += personValue
elif Me[0] == "일식":
    JapanS += personValue
elif Me[0] == "경양식":
    LightwesternS += personValue
elif Me[0] == "뷔페식":
    VuffetS += personValue

if Me[1] == "한식":
    KoreaS -= personValue
elif Me[1] == "중식":
    ChinaS -= personValue
elif Me[1] == "호프/통닭":
    PubS -= personValue
elif Me[1] == "분식":
    SnackbarS -= personValue
elif Me[1] == "양식":
    WesternS -= personValue
elif Me[1] == "기타":
    etcS -= personValue
elif Me[1] == "일식":
    JapanS -= personValue
elif Me[1] == "경양식":
    LightwesternS -= personValue
elif Me[1] == "뷔페식":
    VuffetS -= personValue
    
Dic_between_Food_Score = {"한식" : KoreaS, "중식" : ChinaS, "호프/통닭" : PubS, "분식" : SnackbarS, "양식" : WesternS, "일식" : JapanS, "기타" : etcS, "경양식" : LightwesternS, "뷔페식" : VuffetS}
Ranking = sorted(Dic_between_Food_Score, key = lambda k : Dic_between_Food_Score[k])
Scores = sorted(Dic_between_Food_Score.values())

firstR = Ranking.pop()
secondR = Ranking.pop()
thirdR = Ranking.pop()

firstS = Scores.pop()
secondS = Scores.pop()
thirdS = Scores.pop()

if firstS != 0:
    print("음식 종류 추천 1위 : {} (점수 : {})".format(firstR, firstS))
elif firstS == 0:
    print("모두 0점. 우열을 가릴 수 없습니다.")
if secondS != 0:
    print("음식 종류 추천 2위 : {} (점수 : {})".format(secondR, secondS))
if thirdS != 0:
    print("음식 종류 추천 3위 : {} (점수 : {})".format(thirdR, thirdS))
    
KoreaFs = set()
ChinaFs = set()
PubFs = set()
SnackbarFs = set()
WesternFs = set()
JapanFs = set()
etcFs = set()
LightwesternFs = set()
VuffetFs = set()

for i in range(316):
    if Data.iloc[i,1] == "한식":
        KoreaFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "중식":
        ChinaFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "호프/통닭":
        PubFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "분식":
        SnackbarFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "양식":
        WesternFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "일식":
        JapanFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "기타":
        etcFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "경양식":
        LightwesternFs.add(Data.iloc[i,4])
    elif Data.iloc[i,1] == "뷔페식":
        VuffetFs.add(Data.iloc[i,4])

KoreaF = []
ChinaF = []
PubF = []
SnackbarF = []
WesternF = []
JapanF = []
etcF = []
LightwesternF = []
VuffetF = []

for i in KoreaFs:
    KoreaF.append(i)
for i in ChinaFs:
    ChinaF.append(i)
for i in PubFs:
    PubF.append(i)
for i in SnackbarFs:
    SnackbarF.append(i)
for i in WesternFs:
    WesternF.append(i)
for i in JapanFs:
    JapanF.append(i)
for i in etcFs:
    etcF.append(i)
for i in LightwesternFs:
    LightwesternF.append(i)
for i in VuffetFs:
    VuffetF.append(i)
    
ResName = []

def Search(Choose, df):
    for i in range(316):
        if Choose == df.iloc[i,4]:
            ResName.append(df.iloc[i,0])
    
if firstR == "한식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(KoreaF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "중식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(ChinaF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "호프/통닭":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(PubF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "분식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(SnackbarF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "양식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(WesternF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "기타":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(etcF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "일식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(JapanF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "경양식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(LightwesternF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
elif firstR == "뷔페식":
    Menu = input("1위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(VuffetF))
    Search(Menu , Data)
    print("1위 음식점 추천 ({}): ".format(firstR), ResName)
    
if secondR == "한식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(KoreaF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "중식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(ChinaF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "호프/통닭":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(PubF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "분식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(SnackbarF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "양식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(WesternF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "기타":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(etcF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "일식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(JapanF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "경양식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(LightwesternF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
elif secondR == "뷔페식":
    Menu = input("2위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(VuffetF))
    Search(Menu , Data)
    print("2위 음식점 추천 ({}): ".format(secondR), ResName)
    
if thirdR == "한식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(KoreaF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "중식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(ChinaF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "호프/통닭":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(PubF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "분식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(SnackbarF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "양식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(WesternF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "기타":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(etcF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "일식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(JapanF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "경양식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(LightwesternF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)
elif thirdR == "뷔페식":
    Menu = input("3위 음식 종류의 메뉴를 선택해주세요. 추천 메뉴 : {}".format(VuffetF))
    Search(Menu , Data)
    print("3위 음식점 추천 ({}): ".format(thirdR), ResName)    
    
Address = pd.read_excel('Res_Address.xlsx', encoding = "ANSI")

import folium
import random

name = random.choice(firstR)
k = firstR.index(name)
lat = Address.iloc[k,1]
lng = Address.iloc[k,2]

Map_of_Res = folium.Map(location = [lat, lng], zoom_start = 17)
folium.Marker([lat, lng], popup = name).add_to(Map_of_Res)
Map_of_Res

name = random.choice(secondR)
k = secondR.index(name)
lat = Address.iloc[k,1]
lng = Address.iloc[k,2]

Map_of_Res = folium.Map(location = [lat, lng], zoom_start = 17)
folium.Marker([lat, lng], popup = name).add_to(Map_of_Res)
Map_of_Res

name = random.choice(thirdR)
k = thirdR.index(name)
lat = Address.iloc[k,1]
lng = Address.iloc[k,2]

Map_of_Res = folium.Map(location = [lat, lng], zoom_start = 17)
folium.Marker([lat, lng], popup = name).add_to(Map_of_Res)
Map_of_Res